In [17]:
import tkinter as tk
from tkinter import scrolledtext, messagebox
import spacy
import string
from collections import defaultdict


nlp = spacy.load("en_core_web_sm")


faq_data = {
    "how do i apply for an internship": "You can apply by submitting your resume through our careers page.",
    "what is the duration of the internship": "The internship usually lasts for 3 to 6 months.",
    "do i get a certificate after completion": "Yes, all interns receive a certificate upon successful completion.",
    "is the internship paid": "Currently, our internship program is unpaid.",
    "can i work remotely": "Yes, we offer remote internship options.",
    "what is your name":"My name is Inquario.",
    "who developed you":"I am developed by Junaid Ahamed using Python ",
    "how do i contact support": "You can email our support team at support@inquariopro.com or use the contact form on our website.",
    "what is the internship timings":"Internship can be done in flexible timings as per the interns",
    "When is the deadline":"At the end of this month",
    "How am i valuated":"Your performace will be valuated by the mentors"

}


def preprocess(text):
    doc = nlp(text.lower().translate(str.maketrans('', '', string.punctuation)))
 
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    return tokens


def find_best_match(user_input):
    user_doc = nlp(user_input.lower())
    best_match = None
    max_similarity = 0.0

    for question in faq_data:
        question_doc = nlp(question)
        similarity = user_doc.similarity(question_doc)
        if similarity > max_similarity:
            max_similarity = similarity
            best_match = question

    return best_match if max_similarity > 0.6 else None


class ChatbotApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Inquario 🤖")
        self.root.geometry("600x500")
        self.root.resizable(False, False)
        
        
        self.chat_display = scrolledtext.ScrolledText(
            root, wrap=tk.WORD, width=60, height=20, state='disabled'
        )
        self.chat_display.pack(pady=10)
        
        
        input_frame = tk.Frame(root)
        input_frame.pack(pady=10)
        
        self.user_input = tk.Entry(input_frame, width=50)
        self.user_input.pack(side=tk.LEFT, padx=5)
        self.user_input.bind("<Return>", self.send_message)
        
        send_button = tk.Button(
            input_frame, text="Send", command=self.send_message
        )
        send_button.pack(side=tk.LEFT)
        
        
        self.display_message("Chatbot", "Hi! I'm your Inquario 🤖. Ask me anything or type 'exit' to quit.")
    
    def display_message(self, sender, message):
        self.chat_display.config(state='normal')
        self.chat_display.insert(tk.END, f"{sender}: {message}\n\n")
        self.chat_display.config(state='disabled')
        self.chat_display.see(tk.END)
    
    def send_message(self, event=None):
        user_message = self.user_input.get().strip()
        self.user_input.delete(0, tk.END)
        
        if not user_message:
            return
        
        self.display_message("You", user_message)
        
        if user_message.lower() == 'exit':
            self.display_message("Chatbot", "Goodbye! 👋")
            self.root.after(2000, self.root.destroy)
            return
        
        best_match = find_best_match(user_message)
        
        if best_match:
            self.display_message("Chatbot", faq_data[best_match])
        else:
            self.display_message(
                "Chatbot", 
                "I'm not sure I understand. Try asking about internships, certificates, or remote work."
            )


if __name__ == "__main__":
    root = tk.Tk()
    app = ChatbotApp(root)
    root.mainloop()